In [33]:
from scipy.interpolate import interp1d
import pandas as pd 
from scipy import integrate
import numpy as np
import scipy.stats as ss
import time 

from scipy.stats import norm


In [34]:
#Black and Scholes
def BlackScholes(CallPutFlag,S,X,v,r,T):

    d1 = (np.log(S/X)+(r+v*v/2.)*T)/(v*np.sqrt(T))

    d2 = d1-v*np.sqrt(T)
    if CallPutFlag=="C":
        P = S*norm.cdf(d1) - X*np.exp(-r*T)*norm.cdf(d2)

    else:
        P = -S*norm.cdf(-d1) + X*np.exp(-r*T)*norm.cdf(-d2)
    return P

In [35]:
def ivol(K,IV,Kall):
    if Kall>=K[len(K)-1]:
        Kall=K[len(K)-1]
    if Kall<=K[0]:
        Kall=K[0]
    funy = interp1d(K,IV, kind='cubic', fill_value="extrapolate")
    y=funy(Kall)
  
    if (np.sum(y<0)>0):
        if Kall>=K[len(K)-1]:
            Kall=K[len(K)-1]
        if Kall<=K[0]:
            Kall=K[0]
        funy = interp1d(K,IV, kind='linear', fill_value="extrapolate")
        y=funy(Kall)
    return(y)

In [36]:
def RiskNeutralVolatilitySkewKurt_JVKR(Kvector, IVvector, S0, T, r):
    kmin=.1*S0;
    kmax=3.5*S0;
  
    def V1(K):
        V1=2*(1-np.log(K/S0))*BlackScholes("C",S0, K, ivol(Kvector,IVvector,K), r, T)/np.power(K,2)        
        return(V1)
    def V2(K):
        V2=2*(1+np.log(S0/K))*BlackScholes("P",S0, K, ivol(Kvector,IVvector,K), r, T)/np.power(K,2);        
        return(V2)
    
    def W1(K):
        W1=(6*np.log(K/S0)-3*np.power(np.log(K/S0),2))*BlackScholes("C",S0, K, ivol(Kvector,IVvector,K), r, T)/np.power(K,2);
        return(W1) 
  
    def W2(K):
        W2=(6*np.log(S0/K)+3*np.power(np.log(S0/K),2))*BlackScholes("P",S0, K, ivol(Kvector,IVvector,K), r, T)/np.power(K,2);
        return(W2) 
    
    def X1(K):
        X1=(((12*np.power(np.log(K/S0),2) - 4*np.power(np.log(K/S0),3)))*(BlackScholes("C",S0, K, ivol(Kvector,IVvector,K), r, T))/np.power(K,2));
        return(X1) 
  
    def X2(K):
        X2=(((12*np.power(np.log(S0/K),2) + 4*np.power(np.log(S0/K),3)))*(BlackScholes("P",S0, K, ivol(Kvector,IVvector,K), r, T))/np.power(K,2));
        return(X2) 
    V=integrate.quad(V1,S0,kmax)[0]+integrate.quad(V2,kmin,S0)[0]
    W=integrate.quad(W1,S0,kmax)[0]-integrate.quad(W2,kmin,S0)[0]
    X=integrate.quad(X1,S0,kmax)[0]+integrate.quad(X2,kmin,S0)[0]
    mu=np.exp(r*T)-1-np.exp(r*T)*V/2-np.exp(r*T)*W/6-np.exp(r*T)*X/24;
    #print(V,W,X,mu)
  
    vol=np.sqrt(1/T * V);
    skew=( np.exp(r*T)*W - 3*mu*np.exp(r*T)*V + 2*np.power(mu,3)) / np.power(np.exp(r*T)*V - np.power(mu,2),3/2);
    kurt=( np.exp(r*T)*X - 4*mu*np.exp(r*T)*W + 6*np.exp(r*T)*np.power(mu,2)*V - 3*np.power(mu,4)) / np.power(np.exp(r*T)*V - np.power(mu,2),2);
  
    return([vol,skew,kurt]);



In [37]:
Kvector = [90, 95, 100, 105, 110];
IVvector= [.2, .16, .12, .07, .05];
S0 = 100    
r = 0.07
T = 30 / 365

In [38]:
Data=RiskNeutralVolatilitySkewKurt_JVKR(Kvector, IVvector, S0, T, r)

In [39]:
colnames=["vol","skew","kurt"]
rownames=["Value"]
pd.DataFrame(Data, colnames,rownames)

,Value
vol,0.125560
skew,-1.638387
kurt,6.777459
